# Fake News Classifier Using LSTM

Source: https://www.kaggle.com/c/fake-news/data

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("drive/MyDrive/Fake_news/train.csv")

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.isna().sum(), df.shape

(id           0
 title      558
 author    1957
 text        39
 label        0
 dtype: int64, (20800, 5))

In [5]:
df = df.dropna()

In [6]:
df.shape

(18285, 5)

In [7]:
# Split the data into X and y sets
X = df.drop('label', axis=1)
y = df["label"]

In [8]:
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [9]:
X.shape

(18285, 4)

In [10]:
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [11]:
y.shape

(18285,)

In [12]:
# Import tensorflow and libraries
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM, Dense

In [13]:
# Define Vocabulary size
voc_size = 5000

**Onehot Representation**

In [14]:
messages = X.copy()

In [15]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [16]:
messages.reset_index(inplace=True)

In [17]:
# Import nltk and re for preprocessing
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [20]:
onehot_rep = [one_hot(words, voc_size)for words in corpus]
onehot_rep[0]

[3155, 599, 2535, 4480, 1977, 4024, 4363, 2313, 1612, 592]

**Embedding Representation**

In [21]:
sent_length=20
embedded_docs = pad_sequences(onehot_rep, padding='pre', maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2313 1612  592]
 [   0    0    0 ... 1211  178 1936]
 [   0    0    0 ...  501 1163 1920]
 ...
 [   0    0    0 ... 1062 3315 2643]
 [   0    0    0 ...  505 3729 2849]
 [   0    0    0 ... 3557 1175 4010]]


In [22]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3155,
        599, 2535, 4480, 1977, 4024, 4363, 2313, 1612,  592], dtype=int32)

In [23]:
# Set random seed 
tf.random.set_seed(42)
# Define vector features
embedding_vector_features = 40
# Create a model
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
len(embedded_docs), y.shape

(18285, (18285,))

In [25]:
import numpy as np
X_final = np.asarray(embedded_docs, dtype=np.float)
y_final = np.asarray(y, dtype=np.float)

In [26]:
type(y_final)

numpy.ndarray

In [27]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [28]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.3, random_state=42)

**Model trianing**

In [29]:
# Fit the model
model.fit(X_final, y_final, epochs=10, validation_data=(X_test, y_test), batch_size=64)

Epoch 1/10
286/286 [==============================] - 8s 22ms/step - loss: 0.2945 - accuracy: 0.8637 - val_loss: 0.1444 - val_accuracy: 0.9428
Epoch 2/10
286/286 [==============================] - 6s 21ms/step - loss: 0.1403 - accuracy: 0.9441 - val_loss: 0.0955 - val_accuracy: 0.9652
Epoch 3/10
286/286 [==============================] - 6s 20ms/step - loss: 0.1032 - accuracy: 0.9626 - val_loss: 0.0606 - val_accuracy: 0.9843
Epoch 4/10
286/286 [==============================] - 6s 21ms/step - loss: 0.0741 - accuracy: 0.9747 - val_loss: 0.0446 - val_accuracy: 0.9871
Epoch 5/10
286/286 [==============================] - 6s 20ms/step - loss: 0.0537 - accuracy: 0.9814 - val_loss: 0.0251 - val_accuracy: 0.9934
Epoch 6/10
286/286 [==============================] - 6s 20ms/step - loss: 0.0365 - accuracy: 0.9867 - val_loss: 0.0285 - val_accuracy: 0.9918
Epoch 7/10
286/286 [==============================] - 6s 21ms/step - loss: 0.0221 - accuracy: 0.9930 - val_loss: 0.0101 - val_accuracy: 0.9974

Adding Dropout

In [30]:
from tensorflow.keras.layers import Dropout
# Creating model
embedding_vector_features = 40
model=Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])

**Performance Metrics And Accuracy**

In [31]:
y_pred = model.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [32]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[ 255, 2852],
       [ 286, 2093]])

In [33]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.4279985417426176